In [ ]:
# download semantic drone dataset
!wget https://files.icg.tugraz.at/seafhttp/files/01f38bf8-e12f-417f-9f61-dabe5590ee57/semantic_drone_dataset_semantics_v1.1.zip
!unzip /content/semantic_drone_dataset_semantics_v1.1.zip

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import os
import pickle

In [ ]:
!mkdir /content/drone_people_dataset/

In [ ]:
SEG_FOLDER = '/content/semantic_drone_dataset/training_set/gt/semantic/label_images/'
IMG_FOLDER = '/content/semantic_drone_dataset/training_set/images/'
DICT_PATH = '/content/semantic_drone_dataset/training_set/gt/bounding_box/bounding_boxes/person/imgIdToBBoxArray.p'
RES_PATH = '/content/drone_people_dataset/'

# open dictionary of labels
with open(DICT_PATH, 'rb') as pickleFile:
  bbox_dict = pickle.load(pickleFile)

# open each image from the dataset and save every person as a single image using the bbox annotations
for og_image_path in os.listdir(IMG_FOLDER):
  seg_path = SEG_FOLDER + og_image_path.replace('jpg', 'png')
  og_image = cv2.imread(IMG_FOLDER + og_image_path, 1)
  mask = cv2.imread(seg_path)
  mask[np.where((mask!=[255,22,96]).all(axis=2))] = [0,0,0]  # set every pixel that is not a person to black
  mask[np.where((mask!=[0,0,0]).all(axis=2))] = [1,1,1]  # set every person pixel to 1,1,1 to create mask
  result = np.multiply(og_image, mask)  # apply mask to original image to extract person (multiply element wise by 1)
  for i, bbox in enumerate(bbox_dict[og_image_path.replace('.jpg', '')]):
    img = result[int(bbox[0, 1]):int(bbox[1, 1]), int(bbox[0, 0]):int(bbox[1, 0])]  # slice the image using the bbox
    cv2.imwrite(RES_PATH + og_image_path.replace('.jpg', '') + '_' + str(i).zfill(3) + '.png', img)

In [ ]:
# zip the dataset 
import shutil
shutil.make_archive('drone_people_dataset_black', 'zip', '/content/drone_people_dataset/')

'/content/drone_people_dataset_black.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# save dataset on drive
!cp /content/drone_people_dataset_black.zip /content/drive/Shareddrives/InnLab_Carbonaro/DronePeopleDataset/